# **Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




Load the Data

In [ ]:
data=pd.read_csv('last_satis.csv')

In [ ]:
data

,Unnamed: 0,Tarix,Mağaza,Kart_nomresi,Transaksiya_id,Məhsul_nomresi,Məhsul_adi,Məhsul sayi,Ümumi satış
0,0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...,...
199995,199995,2019-01-09,212,212091,210993,114,Kettle Sensations Siracha Lime 150g,2,9.2
199996,199996,2019-01-29,212,212091,210994,106,Natural ChipCo Hony Soy Chckn175g,2,6.0
199997,199997,2019-04-16,212,212091,210995,61,Smiths Crinkle Cut Chips Chicken 170g,2,5.8
199998,199998,2018-07-28,212,212109,211090,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      200000 non-null  int64         
 1   Tarix           200000 non-null  datetime64[ns]
 2   Mağaza          200000 non-null  int64         
 3   Kart_nomresi    200000 non-null  int64         
 4   Transaksiya_id  200000 non-null  int64         
 5   Məhsul_nomresi  200000 non-null  int64         
 6   Məhsul_adi      200000 non-null  object        
 7   Məhsul sayi     200000 non-null  int64         
 8   Ümumi satış     200000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(6), object(1)
memory usage: 13.7+ MB


In [ ]:
data.describe()

,Unnamed: 0,Tarix,Mağaza,Kart_nomresi,Transaksiya_id,Məhsul_nomresi,Məhsul sayi,Ümumi satış
count,200000.000000,200000,200000.000000,2.000000e+05,2.000000e+05,200000.000000,200000.000000,200000.000000
mean,99999.500000,2018-12-29 23:10:56.784000,132.874300,1.333470e+05,1.329323e+05,56.551910,1.916360,7.339699
min,0.000000,2018-07-01 00:00:00,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.500000
25%,49999.750000,2018-09-30 00:00:00,69.000000,6.911400e+04,6.654675e+04,28.000000,2.000000,5.600000
50%,99999.500000,2018-12-30 00:00:00,128.000000,1.282210e+05,1.320315e+05,56.000000,2.000000,7.400000
75%,149999.250000,2019-03-31 00:00:00,199.000000,1.991970e+05,1.984262e+05,85.000000,2.000000,9.200000
max,199999.000000,2019-06-30 00:00:00,272.000000,2.370961e+06,2.415841e+06,114.000000,200.000000,650.000000
std,57735.171256,NaN,75.755753,7.959404e+04,7.716070e+04,32.812008,0.710336,3.234715


In [ ]:
data.shape

(200000, 9)

# **Data Cleaning**

In [ ]:
data.isnull().sum()

,0
Unnamed: 0,0
Tarix,0
Mağaza,0
Kart_nomresi,0
Transaksiya_id,0
Məhsul_nomresi,0
Məhsul_adi,0
Məhsul sayi,0
Ümumi satış,0


In [ ]:
data['Tarix'].value_counts()

,count
Tarix,
2018-12-24,715
2018-12-23,686
2018-12-19,675
2018-12-22,674
2018-12-18,647
...,...
2019-01-03,495
2018-09-22,492
2018-09-15,490


In [ ]:
print(data.columns.tolist())


['Unnamed: 0', 'Tarix', 'Mağaza', 'Kart_nomresi', 'Transaksiya_id', 'Məhsul_nomresi', 'Məhsul_adi', 'Məhsul sayi', 'Ümumi satış']


In [ ]:
data.drop(columns=['Unnamed: 0', 'Tarix', 'Transaksiya_id', 'Məhsul_nomresi'], inplace=True)

In [ ]:
data

,Mağaza,Kart_nomresi,Məhsul_adi,Məhsul sayi,Ümumi satış
0,1,1000,Natural Chip Compny SeaSalt175g,2,6.0
1,1,1307,CCs Nacho Cheese 175g,3,6.3
2,1,1343,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2,2373,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2,2426,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...
199995,212,212091,Kettle Sensations Siracha Lime 150g,2,9.2
199996,212,212091,Natural ChipCo Hony Soy Chckn175g,2,6.0
199997,212,212091,Smiths Crinkle Cut Chips Chicken 170g,2,5.8
199998,212,212109,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [ ]:
data['Məhsul_adi'].value_counts()

,count
Məhsul_adi,
Kettle Tortilla ChpsHny&Jlpno Chili 150g,2494
Cobs Popd Sea Salt Chips 110g,2483
Kettle Mozzarella Basil & Pesto 175g,2476
Kettle 135g Swt Pot Sea Salt,2465
Tyrrells Crisps Ched & Chives 165g,2465
...,...
WW Crinkle Cut Original 175g,1070
Cheetos Puffs 165g,1067
French Fries Potato Chips 175g,1049


In [ ]:
data

,Mağaza,Kart_nomresi,Məhsul_adi,Məhsul sayi,Ümumi satış,Brand,Description,Weight
0,1,1000,Natural Chip Compny SeaSalt175g,2,6.0,Natural,Chip Compny SeaSalt,175g
1,1,1307,CCs Nacho Cheese 175g,3,6.3,CCs,Nacho Cheese,175g
2,1,1343,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,Smiths,Crinkle Cut Chips Chicken,170g
3,2,2373,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,Smiths,Chip Thinly S/Cream&Onion,175g
4,2,2426,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,Kettle,Tortilla ChpsHny&Jlpno Chili,150g
...,...,...,...,...,...,...,...,...
199995,212,212091,Kettle Sensations Siracha Lime 150g,2,9.2,Kettle,Sensations Siracha Lime,150g
199996,212,212091,Natural ChipCo Hony Soy Chckn175g,2,6.0,Natural,ChipCo Hony Soy Chckn,175g
199997,212,212091,Smiths Crinkle Cut Chips Chicken 170g,2,5.8,Smiths,Crinkle Cut Chips Chicken,170g
199998,212,212109,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8,Doritos,Corn Chip Mexican Jalapeno,150g


In [ ]:
import re


def split_product_name(prod_name):

    weight_search = re.search(r'(\d+[gG])', prod_name)
    weight = weight_search.group(1) if weight_search else None

    brand = prod_name.split()[0]

    prod_wo_weight = re.sub(r'\d+[gG]', '', prod_name).strip()

    parts = prod_wo_weight.split()
    description = ' '.join(parts[1:]).strip() if len(parts) > 1 else None

    return pd.Series([brand, description, weight])

data[['Brand', 'Description', 'Weight']] = data['Məhsul_adi'].apply(split_product_name)

print(data[['Description', 'Weight']].head(10))


                     Description Weight
0            Chip Compny SeaSalt   175g
1                   Nacho Cheese   175g
2      Crinkle Cut Chips Chicken   170g
3      Chip Thinly S Cream Onion   175g
4   Tortilla ChpsHny Jlpno Chili   150g
5  El Paso Salsa Dip Tomato Mild   300g
6     Crinkle Chips Salt Vinegar   330g
7             Waves Sweet Chilli   210g
8     Corn Chip Mexican Jalapeno   150g
9        Waves Sour Cream Chives   210G


In [ ]:
data['Description'].value_counts()

,count
Description,
Cheese,4706
Honey Soy Chicken,3554
Original,3514
Tortilla ChpsHny Jlpno Chili,2494
Popd Sea Salt Chips,2483
...,...
Crinkle Cut Original,1070
Puffs,1067
Fries Potato Chips,1049
